In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

class MinesweeperBoard:
    def __init__(self, width, height, mines):
        self.width = width
        self.height = height
        self.mines = set(mines)
        self.revealed = set()
        self.flagged = set()
    
    def in_bounds(self, x, y):
        return 0 <= x < self.width and 0 <= y < self.height
    
    def adjacent_cells(self, x, y):
        return [(x + dx, y + dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx, dy) != (0, 0) and self.in_bounds(x + dx, y + dy)]

    def count_adjacent_mines(self, x, y):
        return sum(1 for nx, ny in self.adjacent_cells(x, y) if (nx, ny) in self.mines)
    
    def reveal(self, x, y):
        if (x, y) in self.revealed:
            return []
        
        self.revealed.add((x, y))
        
        if (x, y) in self.mines:
            return [(x, y, "mine")]

        count = self.count_adjacent_mines(x, y)
        if count == 0:
            return [(x, y, count)] + [cell for nx, ny in self.adjacent_cells(x, y) for cell in self.reveal(nx, ny)]
        else:
            return [(x, y, count)]

    def is_game_over(self):
        return len(self.revealed) == self.width * self.height - len(self.mines)

def generate_random_board(width, height, num_mines):
    mines = set()
    while len(mines) < num_mines:
        x, y = random.randrange(width), random.randrange(height)
        if (x, y) not in mines:
            mines.add((x, y))
    return MinesweeperBoard(width, height, mines)

def board_to_tensor(board):
    tensor = torch.zeros(10, board.width, board.height)
    for x in range(board.width):
        for y in range(board.height):
            if (x, y) in board.revealed:
                count = board.count_adjacent_mines(x, y)
                tensor[count, x, y] = 1
            elif (x, y) in board.flagged:
                tensor[9, x, y] = 1
            else:
                tensor[8, x, y] = 1
    return tensor.unsqueeze(0)

class MinesweeperCNN(nn.Module):
    def __init__(self):
        super(MinesweeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(10, 25, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(25, 25, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(25, 64, kernel_size=5, padding=2)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = self.output(x)
        return x.squeeze()

def generate_training_data(board, num_samples):
    data = []
    for _ in range(num_samples):
        x, y = random.randrange(board.width), random.randrange(board.height)
        while (x, y) in board.revealed:
            x, y = random.randrange(board.width), random.randrange(board.height)
        tensor = board_to_tensor(board)
        label = torch.tensor(1 if (x, y) in board.mines else 0, dtype=torch.float32)
        data.append((tensor, label))
    return data

def train(model, data, epochs, lr):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        total_loss = 0.0
        for x, y in data:
            optimizer.zero_grad()
            prediction = model(x)
            loss = criterion(prediction, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(data)}")



In [2]:
width, height = 9,9
num_mines = 10
num_samples = 1000
epochs = 20
lr = 0.001

board = generate_random_board(width, height, num_mines)
model = MinesweeperCNN()
data = generate_training_data(board, num_samples)
train(model, data, epochs, lr)

Epoch 1, Loss: 0.40188683312530793
Epoch 2, Loss: 0.37175527444435286
Epoch 3, Loss: 0.36721045257803053
Epoch 4, Loss: 0.37151519799159266
Epoch 5, Loss: 0.36873006518930196
Epoch 6, Loss: 0.36511844652332365
Epoch 7, Loss: 0.36444321222789583
Epoch 8, Loss: 0.36739912287797777
Epoch 9, Loss: 0.3653958429787308
Epoch 10, Loss: 0.36439995154179633
Epoch 11, Loss: 0.3640307130068541
Epoch 12, Loss: 0.36297664123773576
Epoch 13, Loss: 0.3628838225807995
Epoch 14, Loss: 0.36337644785642625
Epoch 15, Loss: 0.3622235392220318
Epoch 16, Loss: 0.36288433173811063
Epoch 17, Loss: 0.3687375410817476
Epoch 18, Loss: 0.35569441359117626
Epoch 19, Loss: 0.355376282133162
Epoch 20, Loss: 0.3552472413778305


In [3]:
model_path = "firstModel.pt"
torch.save(model.state_dict(), model_path)

In [4]:
import numpy as np

def play_minesweeper(model, board, preprocess_fn):
    while not board.is_game_over():
        # Preprocess the board
        input_board = preprocess_fn(board)

        # Predict mine probabilities
        with torch.no_grad():
            input_board = torch.from_numpy(input_board).float().unsqueeze(0)
            probabilities = model(input_board).squeeze().numpy()

        # Choose the cell with the lowest probability of containing a mine
        lowest_prob_idx = np.argmin(probabilities)
        row, col = lowest_prob_idx // board.width, lowest_prob_idx % board.width

        # Reveal the chosen cell
        board.reveal(row, col)

    if board.is_win():
        print("The model won the game!")
    else:
        print("The model lost the game.")


In [15]:
def preprocess_fn(board):
    width = board.width
    height = board.height
    input_board = np.zeros((height, width, 9))

    for i in range(height):
        for j in range(width):
            cell = board.get_cell(i, j)
            if not cell.is_revealed():  # Unrevealed cell
                input_board[i, j, 8] = 1.0
            elif cell.is_mine():  # Mine
                input_board[i, j, 9] = 1.0
            else:  # Revealed cell with count
                count = cell.count_adjacent_mines()
                input_board[i, j, count] = 1.0

    return input_board




In [18]:
play_minesweeper(model, board, preprocess_fn)

NameError: name 'MinesweeperCell' is not defined